# **Geospatial Analysis of Starbucks and McDonald's**

In [8]:
import pandas as pd
import numpy as np
import folium
import geopy.distance

In [9]:
#Read file containing location details
df = pd.read_csv('https://drive.google.com/uc?id=19JERBe0ndadkMPKDFZ0rTEZqdI50zDfk')
df.head(5)

,lat,lng,store,address
0,40.711000,-74.000878,Starbucks,"375 Pearl st, New York, NY 10038"
1,40.707142,-74.004987,Starbucks,"135 John Street, New York, NY 10038"
2,40.708411,-74.007404,Starbucks,"100 William Street, New York, NY 10038"
3,40.711644,-74.006667,Starbucks,"38 Park Row, #4, New York, NY 10038"
4,40.710171,-74.007931,Starbucks,"130 Fulton Street, New York, NY 10038"


In [10]:
#Merge Columns
df['Coordinate']=df[['lat','lng']].apply(tuple, axis=1)
df['Coordinate']

0         (40.711, -74.000878)
1      (40.707142, -74.004987)
2      (40.708411, -74.007404)
3      (40.711644, -74.006667)
4      (40.710171, -74.007931)
                ...           
283    (40.716366, -74.010736)
284    (40.708279, -74.004868)
285    (40.709438, -74.010086)
286     (40.86508, -73.927202)
287    (40.867396, -73.921301)
Name: Coordinate, Length: 288, dtype: object

In [11]:
#New York Empire State Building Lat Long (Looking for any approx center point)
NY_coord = (40.748488, -73.985238)

**1. Compute the distance of the stores**

In [12]:
#Compute distance of every store from city center
distances_km = []

for row in df.itertuples(index=False):
   distances_km.append(geopy.distance.distance(NY_coord, row.Coordinate).km
   )

df['Distance'] = distances_km
df.head(10)

,lat,lng,store,address,Coordinate,Distance
0,40.711000,-74.000878,Starbucks,"375 Pearl st, New York, NY 10038","(40.711, -74.000878)",4.367628
1,40.707142,-74.004987,Starbucks,"135 John Street, New York, NY 10038","(40.707142, -74.004987)",4.885149
2,40.708411,-74.007404,Starbucks,"100 William Street, New York, NY 10038","(40.708411, -74.007404)",4.828401
3,40.711644,-74.006667,Starbucks,"38 Park Row, #4, New York, NY 10038","(40.711644, -74.006667)",4.474066
4,40.710171,-74.007931,Starbucks,"130 Fulton Street, New York, NY 10038","(40.710171, -74.007931)",4.666971
5,40.706316,-74.006582,Starbucks,"80 Pine Street, New York, NY 10005","(40.706316, -74.006582)",5.018288
6,40.712208,-74.008192,Starbucks,"233 Broadway, New York, NY 10279","(40.712208, -74.008192)",4.471204
7,40.708742,-74.009508,Starbucks,"55 Liberty Street, New York, NY 10005","(40.708742, -74.009508)",4.866712
8,40.706033,-74.008348,Starbucks,"60 Wall St, New York, NY 10005","(40.706033, -74.008348)",5.102829
9,40.704817,-74.007231,Starbucks,"99 Wall St., New York, NY 10005","(40.704817, -74.007231)",5.193348


**2.Visualize data on a map**

In [13]:
#Empire State Building coordinates
m = folium.Map(location=[40.748488, -73.985238], zoom_start= 10)

#Place markers for the stores on the map
for i, row in df.iterrows():
  lat = df.at[i, 'lat']
  lng = df.at[i, 'lng']
  store = df.at[i, 'store']

  if store == 'McDonalds':
    color = 'blue'
  else:
    color = 'green'

  folium.Marker(location=[lat,lng], popup=store, icon= folium.Icon(color=color)).add_to(m)

m

**3. Compute number of stores located in a given radius**

In [14]:
#All stores at a distance greater/less than x kms
df[df['Distance'] > 10]

,lat,lng,store,address,Coordinate,Distance
227,40.841350,-73.939822,Starbucks,"4001 Broadway, New York, NY 10032","(40.84135, -73.939822)",11.001594
228,40.841723,-73.941375,Starbucks,"177 Fort Washington Ave , New York, NY 10032","(40.841723, -73.941375)",10.995612
229,40.851000,-73.938421,Starbucks,"803 W 181st St, New York, NY 10033","(40.851, -73.938421)",12.050055
230,40.865721,-73.927287,Starbucks,"4761 Broadway, New York, NY 10040","(40.865721, -73.927287)",13.906793
273,40.835319,-73.944335,McDonalds,3809 BROADWAY,"(40.835319, -73.944335)",10.241912
277,40.842594,-73.938972,McDonalds,4040 BROADWAY,"(40.842594, -73.938972)",11.156082
280,40.850094,-73.936035,McDonalds,4259 BROADWAY,"(40.850094, -73.936035)",12.023104
286,40.865080,-73.927202,McDonalds,208 DYCKMAN ST,"(40.86508, -73.927202)",13.842721
287,40.867396,-73.921301,McDonalds,608 WEST 207TH STREET,"(40.867396, -73.921301)",14.264323


**4. Compute the closest and farthest stores from Empire State Building**



In [15]:
df_farthest = df.iloc[df.groupby('store')['Distance'].agg(pd.Series.idxmax)]
df_closest = df.iloc[df.groupby('store')['Distance'].agg(pd.Series.idxmin)]
df_new = pd.concat([df_farthest, df_closest])

In [16]:
df_new

,lat,lng,store,address,Coordinate,Distance
287,40.867396,-73.921301,McDonalds,608 WEST 207TH STREET,"(40.867396, -73.921301)",14.264323
230,40.865721,-73.927287,Starbucks,"4761 Broadway, New York, NY 10040","(40.865721, -73.927287)",13.906793
256,40.750816,-73.986819,McDonalds,972 6TH AVE,"(40.750816, -73.986819)",0.290966
110,40.748660,-73.985614,Starbucks,"350 Fifth Avenue, New York, NY 10118","(40.74866, -73.985614)",0.037057


In [17]:
#Empire State Building coordinates
m = folium.Map(location=[40.748488, -73.985238], zoom_start= 10)

for i, rown in df_new.iterrows():
  lat = df_new.at[i, 'lat']
  lng = df_new.at[i, 'lng']
  store = df_new.at[i, 'store']

  if store == 'McDonalds':
    color = 'blue'
  else:
    color = 'green'

  folium.Marker(location=[lat,lng], popup=store, icon= folium.Icon(color=color)).add_to(m)

m